<a href="https://colab.research.google.com/github/yuliia-shevchenko137/SimulatedAnnealing/blob/master/ML_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize 
import numpy as np
import networkx as nx

In [ ]:
text = ''
f = open('sample_data/text.txt')
text = f.read()
f.close()
lst = text.split('\n')
lst.remove('')
lst;

Since the psychologist's job is to guide a person through their feelings, it makes sence (at least in my mind, please correct me if I am wrong) to make summary using just the part of the text that a person is saying to psychologist.

For the future, when summary is done, we can give a choise:  
    - to see question and full answer  
    - to see full answer for context  
    - to see the main sentence  

In [ ]:
n = int(len(lst) / 2)
question_answer_dict = {}
just_answers = []
for i in range(n):
    question_answer_dict[i] = (lst[i * 2], lst[i * 2 + 1])
    just_answers.append(lst[i * 2 + 1])
question_answer_dict

{0: ('Психолог: Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу?',
  'Ольга: Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе.'),
 1: ('Психолог: Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?',
  'Ольга: Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не о

Since the sentences are in form "Name: answer", the first part of the sentences (Name: ) can be remover.

Also, additional prep work has to be made.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
    
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
new_text = ''
block = ''
for i in range(len(just_answers)):
    block = just_answers[i].split(": ")[1]
    just_answers[i] = block
    new_text = new_text + " " + block
new_text

' Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе. Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит. Мне словно приходится начинать все с нуля. Деньги немного есть, ээээ ну там на пару месяцев хватит, но нужно искать работу. Не пойму нужно учится на новую специальность или попытаться найти работу менеджером, как раньше? Хотя кому эти менеджеры нужны, ничего толком не работает, наверно повезло людям которые сохранили работу, прям как-то радостно за них. Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладитс

Since we need key bullet points, it makes sence to identify the sentences that have the biggest value using Extractive summarization. 

In [ ]:
sentences = sent_tokenize(new_text)
new_lines = []
for sentence in sentences:
    sentence1 = re.sub(r'[^\w\s]','', sentence) 
    new_lines.append(sentence1)

We will need to create similarity matrix.

In [ ]:
def similarity_matrix(sentences,stop_words):
    n = len(sentences)
    matrix = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            if i!=j:
                sentence1 = word_tokenize(sentences[i].lower())
                sentence2 = word_tokenize(sentences[j].lower())
    
                all_words = list(set(sentence1 + sentence2))
    
                vector1 = [0] * len(all_words)
                vector2 = [0] * len(all_words)
    
                for w in sentence1:
                    if not w in stop_words:
                        vector1[all_words.index(w)]+=1

                for w in sentence2:
                    if not w in stop_words:
                        vector2[all_words.index(w)]+=1
                
                matrix[i][j] = 1-cosine_distance(vector1,vector2)
                
    return matrix

In [ ]:
sim_matrix = similarity_matrix(new_lines,stop_words)

Then, we need to find a rank of each sentence using the Page Rank algorithm.

In [ ]:
graph = nx.from_numpy_array(sim_matrix)
scores = nx.pagerank(graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)

In [ ]:
summary = []
n = 0
while True:
    try:
        n = int(input("How many bullet points do you want to see? \nPlease enter a number: "))
        if n < 3 or n > 7:
            print("The number has to be bigger than 2 and smaller than 8. Try again...")
        else:
            break
    except ValueError:
        print("That was no valid number. Try again...")

top_n = n
for i in range(top_n):
    summary.append(ranked_sentences[i][1])

summary

How many bullet points do you want to see? 
Please enter a number: 7


['Я реально не могу уже я даже иногда думаю ну может просто взять и самой себе жить, а  я не хочу жить одна, мне нравиться наши отношения если бы не это.',
 'Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова.',
 'Пусть даже со мной происходят только хорошие вещи, ну насколько это реально, ну вот даже в плане денег, все есть, понятно накопления на пару месяцев, и работу все равно придется искать, я реально хочу просто определенности, в рамках своей реальности.',
 'Может я просто не понимаю что такое жить с партнером.',
 'А я не хочу ехать одна, я хочу чтобы мы вдвоем поехали, потому что ну что  я там буду делать одна, мне реально надо чтобы мы или вместе были здесь или там.',
 'Я реально хочу в Киев, мне там будет проще собраться с мыслями.',
 'я просто знаю что они бывают и мне уже изначально плохо я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей, не хочу снова все прож

The second sentence ("Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова.") doesn't give much context because we can't understand what necessity she is talking about.  
To avoid situations like that we can give a choise to the user:  
 - to see full answer  
 - to see the question and answer
 - add a few extra sentences

In [ ]:
seq = []
for sentence in summary:
    for i in range(len(just_answers)):
        if just_answers[i].find(sentence) != -1:
            seq.append(i)
seq

[5, 2, 6, 6, 4, 4, 7]

Since some of the key sentences are in the same answer, it makes no sence to double it

In [ ]:
res = list(set(seq))
while True:
    str = input("Do you want to see full answers(A), answers and the questions(Q) or a few extra sentences(S)? Input A, Q, S or No\n")
    if str == "No":
        print("Farewell!")
        break
    if str == "S":
        i = 0
        for sentence in summary:
            i = i + 1
            ind = sentences.index(sentence)
            if ind == 0:
                print(i, ":",  sentences[ind], sentences[ind+1])
            else:
                print(i, ":",  sentences[ind-1],sentences[ind], sentences[ind+1])
        break
    if str == "A":
        print("Key answers:")
        for i in range(len(res)):
            print(i+1, ": ", just_answers[i])
        break
    if str == "Q":
        print("Key answers:")
        for i in range(len(res)):
            print(question_answer_dict[i])
        break
    else:
        print("Try again...")

Do you want to see full answers(A), answers and the questions(Q) or a few extra sentences(S)? Input A, Q, S or No
S
1 : А можно просто было все спокойно сделать вовремя, потому что есть понимание того что вовремя все таки существует, ну или более удачное время, типа чего не пользоваться ситуацией? Я реально не могу уже я даже иногда думаю ну может просто взять и самой себе жить, а  я не хочу жить одна, мне нравиться наши отношения если бы не это. Блин я уже иногда думаю, а если бы мы не были вместе если мне не надо было бы постоянно ждать, где бы я уже была?
2 : Поэтому мне кажется тут у меня что-то вроде иммунитета. Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова. Именно когда живешь в период от нуля до "опять все хочется" тут самое страшное.
3 : Может конечно я свои переживания на него перекладываю, но блин в стране которую постоянно раздирают на части ничего хорошего ждать уже не приходится. Пус

P.S.: To be honest, there is a lot of space for development: 
 - for the answer (S) of the last cell, the other options can be provided
 - some of the questions are too long (Ну он постоянно все на последний момент откладывает, но раньше не было таких критический ситуаций и я могла подождать или делать что я хотела а там уже как было, или мы ссорились ну понятно что потом он принимал решение и позже мне было уже проще я реально никого не напрягала, а там просто все случалось, при чем он всегда понимал что поздно начал шевелится, но мне было даже смешно). For the further development, it might be a good idea to think about a way to split long sentences to a few smaller ones.
 - also, some of key bulet points can represent the same idea, so for potentially further development might be a good idea to group sentences to clussters and choose a representative of each cluster